# Model Comparaison For Anomaly Detection

# Summary

- Introduction
- Before running the notebook
- Config
- Run the evaluation

## Introduction

This is the code used to run the evaluation on all models and all datasets so you can compare the results.
The project output is as follow:
- [stats_for_website.json](https://drive.google.com/file/d/1mBRRRhWLsV2pCC-GMst-9KQ65Owj3jXH/view?usp=sharing) - File used to display the stats on the dataset explorer


- [benchmarks.json](https://drive.google.com/file/d/1cBk0Yd2smMMskUp4U7lPeS2lM2tFPapC/view?usp=sharing) - File used to display the benchmark on the dataset explorer


- In the dataset, duplicate each pictures with it's inference results from efficientad, including the heatmpa. Each file has a `_result` in the filename. Used on the table in the dataset explorer. All the images on this have been reduced to 300x300. You can download the results.zip [here](https://drive.google.com/file/d/1nSSYSPowrj8FwpAAX5cLDZEQldFVf9Uk/view?usp=sharing).

Here is the command used if you are interested: 
```sh
find . -type f \( -name "*.jpg" -o -name "*.png" \) -exec sips -Z 300 {} \;
```

## Before running the notebook

Let's prepare the environement before running the notebook.

Make sure you have run the notebooks `1_baseline`, `2_efficientad`, `3_fomoad` before running this!

Run the following command in the terminal from the **`ai`** folder.

```sh
conda create -p envs/evaluate python=3.11.7 -y
conda activate envs/evaluate

pip install -r requirements/evaluate.txt
```

## Config

In [1]:
import sys  # Import the sys module, required to access system-specific parameters and functions  
sys.path.append('../code/')  # Add the '../code/' directory to the path that Python looks in for files to import  
  
from evaluate import * # Import all contents of the evaluate module

In [2]:

config = {
    "seed": 42,
    "dataset_path": "datasets",
    "baseline" : {
        "cookies_1": "../output/baseline/cookies_1",
        "cookies_2": "../output/baseline/cookies_2",
        "cookies_3": "../output/baseline/cookies_3",
    },
    "efficientad" : {
        "cookies_1": "../output/efficientad/cookies_1",
        "cookies_2": "../output/efficientad/cookies_2",
        "cookies_3": "../output/efficientad/cookies_3",
    },
    "fomoad" : { # Put both mac and linux
        "cookies_1": "../output/fomoad/cookies_1",
        "cookies_1_threshold": 3.7,
        "cookies_2": "../output/fomoad/cookies_2",
        "cookies_2_threshold": 5.2,
        "cookies_3": "../output/fomoad/cookies_3",
        "cookies_3_threshold": 5.0,
    },
}

## Run the evaluation

In [3]:
def load_dataset(subdataset):
     torch.manual_seed(config["seed"])
     np.random.seed(config["seed"])
     random.seed(config["seed"])

     no_anomaly_paths = glob.glob(f"{config['dataset_path']}/{subdataset}/no_anomaly/*.jpg")  # adjust the pattern if needed

     no_anomaly_train_paths, no_anomaly_test_paths = train_test_split(no_anomaly_paths, test_size=0.2, random_state=config["seed"])
     no_anomaly_train_paths,  no_anomaly_val_paths = train_test_split(no_anomaly_train_paths, test_size=0.1, random_state=config["seed"])

     anomaly_lvl_1_paths = glob.glob(f"{config['dataset_path']}/{subdataset}/anomaly_lvl_1/*.jpg")
     anomaly_lvl_2_paths = glob.glob(f"{config['dataset_path']}/{subdataset}/anomaly_lvl_2/*.jpg")
     anomaly_lvl_3_paths = glob.glob(f"{config['dataset_path']}/{subdataset}/anomaly_lvl_3/*.jpg")

     anomaly_lvl_1_test_paths, anomaly_lvl_1_val_paths = train_test_split(anomaly_lvl_1_paths, test_size=0.2, random_state=config["seed"])
     anomaly_lvl_2_test_paths, anomaly_lvl_2_val_paths = train_test_split(anomaly_lvl_2_paths, test_size=0.2, random_state=config["seed"])
     anomaly_lvl_3_test_paths, anomaly_lvl_3_val_paths = train_test_split(anomaly_lvl_3_paths, test_size=0.2, random_state=config["seed"])
     
     
     no_anomaly_labels = ["no_anomaly"] * len(no_anomaly_test_paths)
     no_anomaly_difficulty = ["na"] * len(no_anomaly_test_paths)

     anomaly_1_labels = ["anomaly"] * len(anomaly_lvl_1_test_paths)
     anomaly_1_difficulty = ["easy"] * len(anomaly_lvl_1_test_paths)

     anomaly_2_labels = ["anomaly"] * len(anomaly_lvl_2_test_paths)
     anomaly_2_difficulty = ["medium"] * len(anomaly_lvl_2_test_paths)

     anomaly_3_labels = ["anomaly"] * len(anomaly_lvl_3_test_paths)
     anomaly_3_difficulty = ["hard"] * len(anomaly_lvl_3_test_paths)

     no_anomaly_df = pd.DataFrame({'filename': no_anomaly_test_paths, 'class': no_anomaly_labels, 'difficulty': no_anomaly_difficulty})  

     
     anomaly_1_df = pd.DataFrame({'filename': anomaly_lvl_1_test_paths, 'class': anomaly_1_labels, 'difficulty': anomaly_1_difficulty})  
     anomaly_2_df = pd.DataFrame({'filename': anomaly_lvl_2_test_paths, 'class': anomaly_2_labels, 'difficulty': anomaly_2_difficulty})  
     anomaly_3_df = pd.DataFrame({'filename': anomaly_lvl_3_test_paths, 'class': anomaly_3_labels, 'difficulty': anomaly_3_difficulty})

     combined_df = pd.concat([no_anomaly_df, anomaly_1_df, anomaly_2_df, anomaly_3_df])
     return combined_df.reset_index(drop=True)  

In [4]:
def load_baseline(dataset_name):
    baseline_model = tf.keras.models.load_model(f'{config["baseline"][dataset_name]}/local.keras')
    edge_impulse_model_converted = ImageImpulseRunner(f'{config["baseline"][dataset_name]}/ei.eim')
    edge_impulse_model_converted.init()
        
    return baseline_model, edge_impulse_model_converted

def evaluate_baseline(baseline_model, edge_impulse_model_converted, item):
    cl, time, memory, score = inference_baseline(baseline_model, item["filename"])
    cl_ei, time_ei, memory_ei, score_ei = inference_baseline_ei(edge_impulse_model_converted, item["filename"])
    return cl, time, memory, score, cl_ei, time_ei, memory_ei, score_ei

In [5]:
def load_efficientad(dataset_name):
    config_efficient_ad = {
        "seed": 42,
        "on_gpu": torch.cuda.is_available(),
        
        "out_channels": 384,
        "image_size": 256,
    }
    
    efficientad_models = torch.load(f'{config["efficientad"][dataset_name]}/all_models.pth', map_location=torch.device('cpu'))
    map_normalization = torch.load(f'{config["efficientad"][dataset_name]}/map_normalization.pth', map_location=torch.device('cpu')) 
    
    with open(f'{config["efficientad"][dataset_name]}/best_threshold.pkl', 'rb') as file:  
        best_threshold = pickle.load(file) 
     
    efficientad = EfficientADInference(
        config=config_efficient_ad,
        models=efficientad_models,
        map_normalization=map_normalization,
        threshold=best_threshold,
    )
    
    return efficientad

def evaluate_efficientad(efficientad, item):
    cl, time, memory, score = inference_efficientad(efficientad, item["filename"])
    return cl, time, memory, score

In [6]:
def load_fomoad(dataset_name):
    edge_impulse_model_fomoad = ImageImpulseRunner(f'{config["fomoad"][dataset_name]}/fomoad_runner-mac-x86_64.eim')
    edge_impulse_model_fomoad.init()
     
    return edge_impulse_model_fomoad

def evaluate_fomoad(edge_impulse_model_fomoad, item, threshold):
    cl, time, memory, score = inference_fomoad_ei(edge_impulse_model_fomoad, item["filename"], threshold)
    return cl, time, memory, score


In [7]:
def evaluate_one_dataset(dataset_name):
    print(f"Evaluating {dataset_name}")
    
    # Loading all model
    baseline_model, ei_model_converted = load_baseline(dataset_name)
    efficientad = load_efficientad(dataset_name)
    ei_model_fomoad = load_fomoad(dataset_name)
    
    # Loading dataset
    dataset = load_dataset(dataset_name)
    
    # Looping through the full dataset
    for index, row in tqdm(dataset.iterrows(), total=len(dataset)):
        # Evaluate each model
        cl_baseline, time_baseline, memory_baseline, score_baseline, cl_ei_baseline, time_ei_baseline, memory_ei_baseline, score_ei_baseline = evaluate_baseline(baseline_model, ei_model_converted, row)
        
        cl_efficientad, time_efficientad, memory_efficientad, score_efficientad = evaluate_efficientad(efficientad, row)
        
        cl_fomodad, time_fomoad, memory_fomoad, score_fomoad = evaluate_fomoad(ei_model_fomoad, row, config["fomoad"][f"{dataset_name}_threshold"])
        
        # Update the dataframe to add the different scroes
        dataset.at[index, 'cl_baseline'] = cl_baseline
        dataset.at[index, 'time_baseline'] = time_baseline
        dataset.at[index, 'memory_baseline'] = memory_baseline
        dataset.at[index, 'score_baseline'] = score_baseline
        
        dataset.at[index, 'cl_ei_baseline'] = cl_ei_baseline
        dataset.at[index, 'time_ei_baseline'] = time_ei_baseline
        dataset.at[index, 'memory_ei_baseline'] = memory_ei_baseline
        dataset.at[index, 'score_ei_baseline'] = score_ei_baseline
        
        dataset.at[index, 'cl_efficientad'] = cl_efficientad
        dataset.at[index, 'time_efficientad'] = time_efficientad
        dataset.at[index, 'memory_efficientad'] = memory_efficientad
        dataset.at[index, 'score_efficientad'] = score_efficientad
        
        dataset.at[index, 'cl_fomodad'] = cl_fomodad
        dataset.at[index, 'time_fomoad'] = time_fomoad
        dataset.at[index, 'memory_fomoad'] = memory_fomoad
        dataset.at[index, 'score_fomoad'] = score_fomoad
        
        save_path = row["filename"].replace('.jpg', '_result.jpg')  
        dataset.at[index, 'result_img_path'] = save_path
    return dataset

In [8]:
dataset_1 = evaluate_one_dataset("cookies_1")

Evaluating cookies_1
- Setting seed to 42
- OK - Setting seed to 42 (2.80 ms)



100%|██████████| 100/100 [00:40<00:00,  2.45it/s]


In [9]:
dataset_2 = evaluate_one_dataset("cookies_2")

Evaluating cookies_2
- Setting seed to 42
- OK - Setting seed to 42 (1.18 ms)



100%|██████████| 100/100 [00:43<00:00,  2.29it/s]


In [10]:
dataset_3 = evaluate_one_dataset("cookies_3")

Evaluating cookies_3
- Setting seed to 42
- OK - Setting seed to 42 (1.07 ms)



100%|██████████| 100/100 [00:42<00:00,  2.37it/s]


In [11]:
dataset_3

,filename,class,difficulty,cl_baseline,time_baseline,memory_baseline,score_baseline,cl_ei_baseline,time_ei_baseline,memory_ei_baseline,score_ei_baseline,cl_efficientad,time_efficientad,memory_efficientad,score_efficientad,cl_fomodad,time_fomoad,memory_fomoad,score_fomoad,result_img_path
0,datasets/cookies_3/no_anomaly/20240417_134414.jpg,no_anomaly,na,no_anomaly,205.285788,3760.0,0.629698,no_anomaly,75.807095,-368.0,0.571942,no_anomaly,287.076235,160.0,0.028224,no_anomaly,116.169930,3072.0,3.977825,datasets/cookies_3/no_anomaly/20240417_134414_...
1,datasets/cookies_3/no_anomaly/20240417_134257.jpg,no_anomaly,na,no_anomaly,60.723066,48.0,0.747607,no_anomaly,37.753105,240.0,0.736483,no_anomaly,291.661978,48.0,0.010483,no_anomaly,35.536766,3200.0,2.709480,datasets/cookies_3/no_anomaly/20240417_134257_...
2,datasets/cookies_3/no_anomaly/20240417_134101.jpg,no_anomaly,na,anomaly,37.206888,208.0,0.677503,no_anomaly,36.581278,1280.0,0.567319,no_anomaly,321.985006,0.0,0.025547,no_anomaly,37.643194,3168.0,4.017349,datasets/cookies_3/no_anomaly/20240417_134101_...
3,datasets/cookies_3/no_anomaly/20240417_134452.jpg,no_anomaly,na,no_anomaly,41.023016,112.0,0.697850,no_anomaly,30.738115,208.0,0.583880,no_anomaly,290.608883,0.0,0.013774,no_anomaly,35.781145,4016.0,4.191556,datasets/cookies_3/no_anomaly/20240417_134452_...
4,datasets/cookies_3/no_anomaly/20240417_134334.jpg,no_anomaly,na,no_anomaly,39.387941,0.0,0.642211,anomaly,31.686783,192.0,0.502757,no_anomaly,293.304920,0.0,0.008547,no_anomaly,35.309076,4192.0,3.373817,datasets/cookies_3/no_anomaly/20240417_134334_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,datasets/cookies_3/anomaly_lvl_3/20240417_1406...,anomaly,hard,no_anomaly,40.112257,0.0,0.667141,no_anomaly,31.785965,0.0,0.570048,anomaly,306.662083,3152.0,0.075944,anomaly,36.483049,-63008.0,5.378742,datasets/cookies_3/anomaly_lvl_3/20240417_1406...
96,datasets/cookies_3/anomaly_lvl_3/20240417_1401...,anomaly,hard,anomaly,42.176008,-30576.0,0.566778,anomaly,32.583952,0.0,0.825505,anomaly,295.277834,-37664.0,0.061785,anomaly,38.079023,16.0,8.081696,datasets/cookies_3/anomaly_lvl_3/20240417_1401...
97,datasets/cookies_3/anomaly_lvl_3/20240417_1405...,anomaly,hard,anomaly,41.497946,1072.0,0.652064,anomaly,37.628889,0.0,0.857419,anomaly,292.452812,-21984.0,0.062294,anomaly,35.465240,0.0,6.917777,datasets/cookies_3/anomaly_lvl_3/20240417_1405...
98,datasets/cookies_3/anomaly_lvl_3/20240417_1405...,anomaly,hard,anomaly,37.263870,288.0,0.627439,anomaly,32.335281,0.0,0.783686,anomaly,301.133871,-27680.0,0.033368,anomaly,35.921335,0.0,12.952106,datasets/cookies_3/anomaly_lvl_3/20240417_1405...


In [12]:
def get_score(dataset, name):  
    row = [name]  
    
    # For Baseline  
    f1_all_baseline = f1_score(dataset['class'], dataset['cl_baseline'], pos_label='anomaly')  
    
    f1_all_baseline_na = f1_score(dataset[dataset["difficulty"] == "na"]['class'], dataset[dataset["difficulty"] == "na"]['cl_baseline'], pos_label='no_anomaly')  
    f1_all_baseline_lvl1 = f1_score(dataset[dataset["difficulty"] == "easy"]['class'], dataset[dataset["difficulty"] == "easy"]['cl_baseline'], pos_label='anomaly')  
    f1_all_baseline_lvl2 = f1_score(dataset[dataset["difficulty"] == "medium"]['class'], dataset[dataset["difficulty"] == "medium"]['cl_baseline'], pos_label='anomaly')  
    f1_all_baseline_lvl3 = f1_score(dataset[dataset["difficulty"] == "hard"]['class'], dataset[dataset["difficulty"] == "hard"]['cl_baseline'], pos_label='anomaly')  
    row.extend([name, f1_all_baseline, f1_all_baseline_na, f1_all_baseline_lvl1, f1_all_baseline_lvl2, f1_all_baseline_lvl3,  
                dataset['time_baseline'].mean(), dataset['memory_baseline'].mean()])  
    # For Baseline EI  
    f1_all_baseline_ei = f1_score(dataset['class'], dataset['cl_ei_baseline'], pos_label='anomaly')  
    f1_all_baseline_ei_na = f1_score(dataset[dataset["difficulty"] == "na"]['class'], dataset[dataset["difficulty"] == "na"]['cl_ei_baseline'], pos_label='no_anomaly')  
    f1_all_baseline_ei_lvl1 = f1_score(dataset[dataset["difficulty"] == "easy"]['class'], dataset[dataset["difficulty"] == "easy"]['cl_ei_baseline'], pos_label='anomaly')  
    f1_all_baseline_ei_lvl2 = f1_score(dataset[dataset["difficulty"] == "medium"]['class'], dataset[dataset["difficulty"] == "medium"]['cl_ei_baseline'], pos_label='anomaly')  
    f1_all_baseline_ei_lvl3 = f1_score(dataset[dataset["difficulty"] == "hard"]['class'], dataset[dataset["difficulty"] == "hard"]['cl_ei_baseline'], pos_label='anomaly')  
    row.extend([name, f1_all_baseline_ei, f1_all_baseline_ei_na, f1_all_baseline_ei_lvl1, f1_all_baseline_ei_lvl2, f1_all_baseline_ei_lvl3,  
                dataset['time_ei_baseline'].mean(), dataset['memory_ei_baseline'].mean()])  
  
    # For EfficientAD  
    f1_all_efficientad = f1_score(dataset['class'], dataset['cl_efficientad'], pos_label='anomaly')  
    f1_all_efficientad_na = f1_score(dataset[dataset["difficulty"] == "na"]['class'], dataset[dataset["difficulty"] == "na"]['cl_efficientad'], pos_label='no_anomaly')  
    f1_all_efficientad_lvl1 = f1_score(dataset[dataset["difficulty"] == "easy"]['class'], dataset[dataset["difficulty"] == "easy"]['cl_efficientad'], pos_label='anomaly')  
    f1_all_efficientad_lvl2 = f1_score(dataset[dataset["difficulty"] == "medium"]['class'], dataset[dataset["difficulty"] == "medium"]['cl_efficientad'], pos_label='anomaly')  
    f1_all_efficientad_lvl3 = f1_score(dataset[dataset["difficulty"] == "hard"]['class'], dataset[dataset["difficulty"] == "hard"]['cl_efficientad'], pos_label='anomaly')  
    row.extend([name, f1_all_efficientad, f1_all_efficientad_na, f1_all_efficientad_lvl1, f1_all_efficientad_lvl2, f1_all_efficientad_lvl3,  
                dataset['time_efficientad'].mean(), dataset['memory_efficientad'].mean()])  
  
    # For FOMOAD  
    f1_all_fomoad = f1_score(dataset['class'], dataset['cl_fomodad'], pos_label='anomaly')  
    f1_all_fomoad_na = f1_score(dataset[dataset["difficulty"] == "na"]['class'], dataset[dataset["difficulty"] == "na"]['cl_fomodad'], pos_label='no_anomaly')  
    f1_all_fomoad_lvl1 = f1_score(dataset[dataset["difficulty"] == "easy"]['class'], dataset[dataset["difficulty"] == "easy"]['cl_fomodad'], pos_label='anomaly')  
    f1_all_fomoad_lvl2 = f1_score(dataset[dataset["difficulty"] == "medium"]['class'], dataset[dataset["difficulty"] == "medium"]['cl_fomodad'], pos_label='anomaly')  
    f1_all_fomoad_lvl3 = f1_score(dataset[dataset["difficulty"] == "hard"]['class'], dataset[dataset["difficulty"] == "hard"]['cl_fomodad'], pos_label='anomaly')  
    row.extend([name, f1_all_fomoad, f1_all_fomoad_na, f1_all_fomoad_lvl1, f1_all_fomoad_lvl2, f1_all_fomoad_lvl3,  
                dataset['time_fomoad'].mean(), dataset['memory_fomoad'].mean()])  
  
    return row

benchmarks = pd.DataFrame(columns=["Dataset", 'Baseline Dataset', 'Baseline F1 TEST', 'Baseline F1 NO ANOMALY TEST', 'Baseline F1 EASY TEST', 'Baseline F1 MEDIUM TEST', 'Baseline F1 HARD TEST', 'Baseline Time (ms)', 'Baseline RAM (KB)',  
                                   'Baseline EI Dataset', 'Baseline EI F1 TEST', 'Baseline EI F1 NO ANOMALY TEST', 'Baseline EI F1 EASY TEST', 'Baseline EI F1 MEDIUM TEST', 'Baseline EI F1 HARD TEST', 'Baseline EI Time (ms)', 'Baseline EI RAM (KB)',  
                                   'EfficientAD Dataset', 'EfficientAD F1 TEST', 'EfficientAD F1 NO ANOMALY TEST', 'EfficientAD F1 EASY TEST', 'EfficientAD F1 MEDIUM TEST', 'EfficientAD F1 HARD TEST', 'EfficientAD Time (ms)', 'EfficientAD RAM (KB)',  
                                   'FOMOAD Dataset', 'FOMOAD F1 TEST', 'FOMOAD F1 NO ANOMALY TEST', 'FOMOAD F1 EASY TEST', 'FOMOAD F1 MEDIUM TEST', 'FOMOAD F1 HARD TEST', 'FOMOAD Time (ms)', 'FOMOAD RAM (KB)'])  
  
# Assuming dataset1, dataset2, dataset3 are your actual datasets  
results_dataset1 = pd.DataFrame([get_score(dataset_1, "cookies_1")], columns=benchmarks.columns)  
results_dataset2 = pd.DataFrame([get_score(dataset_2, "cookies_2")], columns=benchmarks.columns)  
results_dataset3 = pd.DataFrame([get_score(dataset_3, "cookies_3")], columns=benchmarks.columns)  

def dataframe_to_dict(df):  
    return df.to_dict(orient='records')[0]  

results = {  
    "cookies_1": dataframe_to_dict(results_dataset1),  
    "cookies_2": dataframe_to_dict(results_dataset2),  
    "cookies_3": dataframe_to_dict(results_dataset3)  
}

benchmarks = pd.concat([benchmarks, results_dataset1, results_dataset2, results_dataset3], ignore_index=True)
import json
with open('../output/benchmarks.json', 'w') as outfile:  
    outfile.write(json.dumps(results))

/var/folders/x_/0z24g8110_n09vlvhxmrh0_w0000gp/T/ipykernel_94509/1648641828.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  benchmarks = pd.concat([benchmarks, results_dataset1, results_dataset2, results_dataset3], ignore_index=True)


In [13]:
results_dataset1

,Dataset,Baseline Dataset,Baseline F1 TEST,Baseline F1 NO ANOMALY TEST,Baseline F1 EASY TEST,Baseline F1 MEDIUM TEST,Baseline F1 HARD TEST,Baseline Time (ms),Baseline RAM (KB),Baseline EI Dataset,...,EfficientAD Time (ms),EfficientAD RAM (KB),FOMOAD Dataset,FOMOAD F1 TEST,FOMOAD F1 NO ANOMAL TEST,FOMOAD F1 EASY TEST,FOMOAD F1 MEDIUM TEST,FOMOAD F1 HARD TEST,FOMOAD Time (ms),FOMOAD RAM (KB)
0,cookies_1,cookies_1,0.865248,1.0,1.0,1.0,0.095238,39.690981,436.64,cookies_1,...,299.973309,9296.64,cookies_1,0.954248,1.0,1.0,1.0,0.787879,35.132482,2337.6


In [14]:
new_columns = pd.MultiIndex.from_tuples([  
    ('', 'Dataset'),                                # Common dataset column  
    ('Baseline', 'Dataset'),  
    ('Baseline', 'F1 TEST'), ('Baseline', 'F1 NO ANOMALY'), ('Baseline', 'F1 EASY'), ('Baseline', 'F1 MEDIUM'), ('Baseline', 'F1 HARD'), ('Baseline', 'Time (ms)'), ('Baseline', 'RAM (KB)'),  
    ('Baseline EI', 'Dataset'),  
    ('Baseline EI', 'F1 TEST'), ('Baseline EI', 'F1 NO ANOMALY'), ('Baseline EI', 'F1 EASY'), ('Baseline EI', 'F1 MEDIUM'), ('Baseline EI', 'F1 HARD'), ('Baseline EI', 'Time (ms)'), ('Baseline EI', 'RAM (KB)'),  
    ('EfficientAD', 'Dataset'),  
    ('EfficientAD', 'F1 TEST'), ('EfficientAD', 'F1 NO ANOMALY'), ('EfficientAD', 'F1 EASY'), ('EfficientAD', 'F1 MEDIUM'), ('EfficientAD', 'F1 HARD'), ('EfficientAD', 'Time (ms)'), ('EfficientAD', 'RAM (KB)'),  
    ('FOMOAD', 'Dataset'),  
    ('FOMOAD', 'F1 TEST'), ('FOMOAD', 'F1 NO ANOMALY'), ('FOMOAD', 'F1 EASY'), ('FOMOAD', 'F1 MEDIUM'), ('FOMOAD', 'F1 HARD'), ('FOMOAD', 'Time (ms)'), ('FOMOAD', 'RAM (KB)')  
])  
benchmarks.columns = new_columns  

# Show styled DataFrame  
print("BASELINE")
display(benchmarks.loc[:, 'Baseline'])
print("Baseline EI")
display(benchmarks.loc[:, 'Baseline EI'])  
print("EfficientAD")
display(benchmarks.loc[:, 'EfficientAD'])
print("FOMO AD")
display(benchmarks.loc[:, 'FOMOAD'])

BASELINE


,Dataset,F1 TEST,F1 NO ANOMALY,F1 EASY,F1 MEDIUM,F1 HARD,Time (ms),RAM (KB)
0,cookies_1,0.865248,1.000000,1.0,1.0,0.095238,39.690981,436.64
1,cookies_2,0.888889,1.000000,1.0,1.0,0.333333,43.644302,125.12
2,cookies_3,0.917197,0.857143,1.0,1.0,0.750000,42.710402,-192.32


Baseline EI


,Dataset,F1 TEST,F1 NO ANOMALY,F1 EASY,F1 MEDIUM,F1 HARD,Time (ms),RAM (KB)
0,cookies_1,0.875000,0.974359,1.0,1.0,0.260870,31.956229,476.80
1,cookies_2,0.954248,1.000000,1.0,1.0,0.787879,33.593299,17.92
2,cookies_3,0.941935,0.947368,1.0,1.0,0.787879,33.523099,244.48


EfficientAD


,Dataset,F1 TEST,F1 NO ANOMALY,F1 EASY,F1 MEDIUM,F1 HARD,Time (ms),RAM (KB)
0,cookies_1,0.904110,1.0,1.0,1.000000,0.461538,299.973309,9296.64
1,cookies_2,0.888889,1.0,1.0,0.918919,0.518519,318.657351,2638.08
2,cookies_3,0.993711,1.0,1.0,1.000000,0.974359,305.910184,2326.72


FOMO AD


,Dataset,F1 TEST,F1 NO ANOMALY,F1 EASY,F1 MEDIUM,F1 HARD,Time (ms),RAM (KB)
0,cookies_1,0.954248,1.0,1.0,1.0,0.787879,35.132482,2337.60
1,cookies_2,1.000000,1.0,1.0,1.0,1.000000,38.245327,129.12
2,cookies_3,0.993711,1.0,1.0,1.0,0.974359,38.416507,956.32


In [15]:
benchmarks.to_csv('../output/benchmarks.csv', index=False)  

In [16]:
def dataframe_to_json(df):
    # Transform DataFrame to the specified JSON structure  
    transformed_data = {  
        'baseline': df[['filename', 'class', 'difficulty', 'cl_baseline', 'score_baseline', 'time_baseline']].rename(columns={  
            'filename': 'i',  
            'class': 'c',  
            'difficulty': 'd',  
            'cl_baseline': 'cl',  
            'score_baseline': 's',  
            'time_baseline': 't'  
        }).to_dict(orient='records'),  
        'baseline-ei': df[['filename', 'class', 'difficulty', 'cl_ei_baseline', 'score_ei_baseline', 'time_ei_baseline']].rename(columns={  
             'filename': 'i',  
            'class': 'c',  
            'difficulty': 'd',  
            'cl_ei_baseline': 'cl',  
            'score_ei_baseline': 's',  
            'time_ei_baseline': 't'  
        }).to_dict(orient='records'),  
        'efficientad': df[['filename', 'class', 'difficulty', 'cl_efficientad', 'score_efficientad', 'time_efficientad', 'result_img_path']].rename(columns={  
            'filename': 'i',  
            'class': 'c',  
            'difficulty': 'd',  
            'cl_efficientad': 'cl',  
            'score_efficientad': 's',  
            'time_efficientad': 't',  
            'result_img_path': 'r'  
        }).to_dict(orient='records'),  
        'fomoad': df[['filename', 'class', 'difficulty', 'cl_fomodad', 'score_fomoad', 'time_fomoad']].rename(columns={  
            'filename': 'i',  
            'class': 'c',  
            'difficulty': 'd',  
            'cl_fomodad': 'cl',  
            'score_fomoad': 's',  
            'time_fomoad': 't'  
        }).to_dict(orient='records')
    }
    
    return  transformed_data

t1 =  dataframe_to_json(dataset_1)
t2 =  dataframe_to_json(dataset_2)
t3 =  dataframe_to_json(dataset_3)

stats_for_website = {
    "cookies_1": t1,
    "cookies_2": t2,
    "cookies_3": t3,
}
import json  
with open('../output/stats_for_website.json', 'w') as outfile:  
    json.dump(stats_for_website, outfile)  